In [1]:
import os
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

from scipy.stats import linregress
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
pd.options.display.max_columns = 80

os.chdir('/Users/MatthewBarnette/data_science/house_prices/data')
#os.chdir('C:/Users/spike/OneDrive/data_science/house_prices/data')

train = pd.read_csv('train.csv',index_col='Id')

# Train Data

To start off I want to make a correlation matrix, however categorical data will not work well. So I'm going to use label encoding to convert it so that it is at least numeric, though not continuous.

In [2]:
object_columns = train.select_dtypes(include=['object']).columns.values

for column in object_columns:
    le.fit(train[column].astype(str))
    train[column] = le.transform(train[column].astype(str))

I'm creating the correlation matrix and filtering it down to a correlation of greater than .5 as there are 80 columns in this dataset and I'm certain some of them are not as relavent as others. As SalePrice is the variable of interest I am mainly looking at the correlation of that column.

In [3]:
train_corr = train.corr(method='pearson')
train_corr = train_corr[(train_corr.SalePrice >= .5)]
train_corr = train_corr[train_corr.index]

From there I take the correlation matrix and convert it into a heatmap for ease of viewing.

In [4]:
train_heatmap = [go.Heatmap(z=train_corr.values.tolist(),
                            x=train_corr.columns.values,
                            y=train_corr.index,
                            colorscale='Viridis')]

py.iplot(train_heatmap,filename='housing_data_corr_matrix')

Now we have some correlation data, but what does our sale price data look like.

In [5]:
train.SalePrice.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

Looking at the below histogram we can see that the prices are positively skewed with a few outliers.

In [6]:
salehist = [go.Histogram(x=train.SalePrice)]
py.iplot(salehist,file_name='sale_price_histogram',bin=20000)

Another thing which could be useful is taking a look at those items which are closely correlated in relations to the SalePrice variable

In [7]:
qual_sale = [go.Box(x=train['OverallQual'],y=train['SalePrice'])]

py.iplot(qual_sale,filename='box_quality_sale')

In [8]:
bath_sale = [go.Box(x=train['FullBath'],y=train['SalePrice'])]

py.iplot(bath_sale,filename='box_fullbath_sale')

In [9]:
year_built = [go.Box(x=train['YearBuilt'],y=train['SalePrice'])]

py.iplot(year_built,filename='box_yearbuilt_sale')

In [10]:
year_remod = [go.Box(x=train['YearRemodAdd'],y=train['SalePrice'])]

py.iplot(year_remod,filename='box_yearremod_sale')

In [11]:
cars_sale = [go.Box(x=train['GarageCars'],y=train['SalePrice'])]

py.iplot(cars_sale,filename='box_garagecar_sale')

In [12]:
slope, intercept, r_value, p_value, std_err = linregress(train['GarageArea'],train['SalePrice'])
bf_line = (train['GarageArea'] * slope) + intercept
garagearea_sale = go.Scatter(x=train['GarageArea'],y=train['SalePrice'],mode='markers',name = 'Garage Area')
garagearea_sale_line = go.Scatter(x=train['GarageArea'],y=bf_line,mode='lines',name='Best Fit')

py.iplot([garagearea_sale,garagearea_sale_line],filename='scatter_garagearea_sale')

In [13]:
slope, intercept, r_value, p_value, std_err = linregress(train['1stFlrSF'],train['SalePrice'])
bf_line = (train['1stFlrSF'] * slope) + intercept
fstflrsf_sale = go.Scatter(x=train['1stFlrSF'],y=train['SalePrice'],mode='markers',name='First Floor Sq Ft')
fstflrsf_sale_line = go.Scatter(x=train['1stFlrSF'],y=bf_line,mode='lines',name='Best Fit')

py.iplot([fstflrsf_sale,fstflrsf_sale_line],filename='scatter_fstflrsf_sale')

In [14]:
slope, intercept, r_value, p_value, std_err = linregress(train['TotalBsmtSF'],train['SalePrice'])
bf_line = (train['TotalBsmtSF'] * slope) + intercept
bsmtsf_sale = go.Scatter(x=train['TotalBsmtSF'],y=train['SalePrice'],mode='markers',name='Basement Sq Ft')
bsmtsf_sale_line = go.Scatter(x=train['TotalBsmtSF'],y=bf_line,mode='lines',name='Best Fit')

py.iplot([bsmtsf_sale,bsmtsf_sale_line],filename='scatter_bsmtsf_sale')

In [15]:
slope, intercept, r_value, p_value, std_err = linregress(train['GrLivArea'],train['SalePrice'])
bf_line = (train['GrLivArea'] * slope) + intercept
grlivarea_sale = go.Scatter(x=train['GrLivArea'],y=train['SalePrice'],mode='markers',name='Ground Living Area')
grlivarea_sale_line = go.Scatter(x=train['GrLivArea'],y=bf_line,mode='lines',name='Best Fit')

py.iplot([grlivarea_sale,grlivarea_sale_line],filename='scatter_grlivarea_sale')

One thing that I notice throughout most of the scatter plots is there are two points that continually seems to be outliers with prices of 160k and 184.75k even though in several areas it has quite a large square footage which seems to be correlated with price.

In [16]:
train[(train.SalePrice.isin([160000,184750])) & (train.GrLivArea.isin([5642,4676]))]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
524,60,3,130.0,40094,1,2,0,0,0,4,0,7,4,4,0,5,10,5,2007,2008,3,1,5,5,3,762.0,0,4,2,0,3,1,2,2260,5,0,878,3138,1,0,1,4,3138,1538,0,4676,1,0,3,1,3,1,0,11,6,1,2,3,2007.0,0,3,884,4,4,2,208,406,0,0,0,0,3,4,4,0,10,2007,6,5,184750
1299,60,3,313.0,63887,1,2,2,0,0,0,0,7,1,2,0,5,10,5,2008,2008,3,0,11,12,3,796.0,0,4,2,0,3,1,2,5644,5,0,466,6110,1,0,1,4,4692,950,0,5642,2,0,2,1,3,1,0,12,6,3,2,1,2008.0,0,2,1418,4,4,2,214,292,0,0,0,480,2,4,4,0,1,2008,6,5,160000


# Test Data

One other thing we should do is look through our data to see if there is anything that needs cleaning up. First let's get our columns we'll be looking through based on our correlation matrix

In [17]:
corr_columns = train_corr.columns.values

In [18]:
train = train[corr_columns]

In [21]:
train.isnull().any()

OverallQual     False
YearBuilt       False
YearRemodAdd    False
TotalBsmtSF     False
1stFlrSF        False
GrLivArea       False
FullBath        False
TotRmsAbvGrd    False
GarageCars      False
GarageArea      False
SalePrice       False
dtype: bool

The training data seems to be clean of null values, now let us take a look at the test data

In [23]:
test = pd.read_csv('test.csv')

In [25]:
test = test[corr_columns[0:-1]]

In [26]:
test.isnull().any()

OverallQual     False
YearBuilt       False
YearRemodAdd    False
TotalBsmtSF      True
1stFlrSF        False
GrLivArea       False
FullBath        False
TotRmsAbvGrd    False
GarageCars       True
GarageArea       True
dtype: bool

And here we see that the TotalBsmtSF, GarageCars and GarageArea variables have null values in them. When we load this data we will replace these values with a 0.